In [6]:
!pip3 install torch  --index-url https://download.pytorch.org/whl/cu128


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu128


In [8]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

False
0
No GPU found


Prerequisites

In [10]:
pip install transformers datasets torch accelerate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


1. Preparing Dataset

In [11]:
from datasets import load_dataset

dataset = load_dataset("tiny_shakespeare", trust_remote_code=True)


In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})

In [13]:
dataset["test"][0]

{'text': "rance ta'en\nAs shall with either part's agreement stand?\n\nBAPTISTA:\nNot in my house, Lucentio; for, you know,\nPitchers have ears, and I have many servants:\nBesides, old Gremio is hearkening still;\nAnd happily we might be interrupted.\n\nTRANIO:\nThen at my lodging, an it like you:\nThere doth my father lie; and there, this night,\nWe'll pass the business privately and well.\nSend for your daughter by your servant here:\nMy boy shall fetch the scrivener presently.\nThe worst is this, that, at so slender warning,\nYou are like to have a thin and slender pittance.\n\nBAPTISTA:\nIt likes me well. Biondello, hie you home,\nAnd bid Bianca make her ready straight;\nAnd, if you will, tell what hath happened,\nLucentio's father is arrived in Padua,\nAnd how she's like to be Lucentio's wife.\n\nBIONDELLO:\nI pray the gods she may with all my heart!\n\nTRANIO:\nDally not with the gods, but get thee gone.\nSignior Baptista, shall I lead the way?\nWelcome! one mess is like to be yo

2. Tokenization

In [14]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [17]:
def tokenize_function(examples):
    return tokenizer(examples['text'],
                     truncation=True,
                     max_length=512,
                     padding='max_length')

tokenized_data = dataset.map(tokenize_function, batched=True)

3. Model Initialization

In [18]:
pip install hf_xet

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 30.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')


4. Training Setup

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
)

5. Start Training

In [21]:
trainer.train()

C:\Program Files\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.